In [229]:
import numpy as np
import matplotlib.pyplot as plt
import torch

def plot_f_3d(f, xlim=(0.0, 1.0), ylim=(0.0, 1.0), n=200, title="f(x,y)"):
    """
    Rysuje funkcję f(x,y) jako powierzchnię 3D na siatce [xlim]×[ylim].

    f: callable, powinno przyjmować (X, Y) jako tablice (np.meshgrid) i zwracać Z tej samej wielkości.
       Jeśli f działa tylko na skalarach, spróbujemy je zwektoryzować.
    """
    x = np.linspace(xlim[0], xlim[1], n)
    y = np.linspace(ylim[0], ylim[1], n)
    X, Y = np.meshgrid(x, y)

    try:
        Z = f(X, Y)
    except Exception:
        Z = np.vectorize(f)(X, Y)

    Z = np.asarray(Z)

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection="3d")
    ax.plot_surface(X, Y, Z, linewidth=0, antialiased=True)

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("f(x,y)")
    ax.set_title(title)
    plt.tight_layout()
    plt.show()


In [230]:
import numpy as np
import plotly.graph_objects as go

def plot_f_3d_interactive(f, xlim=(0.0, 1.0), ylim=(0.0, 1.0), n=200, title="f(x,y)"):
    """
    Interaktywny 3D surface w notebooku (Plotly).
    f powinno działać na (X, Y) jako macierze; jeśli nie, to wektoryzujemy.
    """
    x = np.linspace(xlim[0], xlim[1], n)
    y = np.linspace(ylim[0], ylim[1], n)
    X, Y = np.meshgrid(x, y)

    try:
        Z = f(X, Y)
    except Exception:
        Z = np.vectorize(f)(X, Y)

    Z = np.asarray(Z)

    fig = go.Figure(
        data=go.Surface(x=X, y=Y, z=Z)
    )
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="f(x,y)",
        ),
        margin=dict(l=0, r=0, b=0, t=40),
    )
    fig.show()


In [231]:
def f_1(x, y):
        f_target = 0.853
        x, y = torch.tensor(x), torch.tensor(y)
        pi_target = torch.tensor(f_target * torch.pi)
        loss = ((y * torch.sin(pi_target) + x * torch.cos(pi_target))
            + 2 * torch.abs((0.5 - y) ** 2 + (0.5 - x) ** 2 - 0.25))
        return loss.numpy(force=True)

plot_f_3d_interactive(f_1, title="f_1(x,y)")

In [234]:
import numpy as np

def make_fold_function_through_y(s, side="right", R=0.5, cx=0.5, cy=0.5):
    """
    Zwraca funkcję f(x,y) = | (x-cx)cos(theta) + (y-cy)sin(theta) |,
    której 'zagięcie' (linia gdzie wyrażenie w |.| = 0) przechodzi przez punkt
    na okręgu o środku (cx,cy) i promieniu R z zadanym y = s.

    side: "right" albo "left" wybiera punkt przecięcia na prawej/lewej stronie okręgu.
    """
    dy = s - 0.5
    dx_abs = -np.sqrt(0.5**2 - dy**2)
    theta = np.arctan2(dx_abs, -dy)

    def f(x, y):
        return (
                0*(x - 0.5)**2 * np.cos(theta) - 1*(y - 0.5)**2 * np.sin(theta) +
                # np.abs((x - 0.5) * np.cos(theta) - (y - 0.5) * np.sin(theta)) +
         x*np.sin(theta) + y*np.cos(theta) +
         -2 * ((0.5 - y) ** 2 + (0.5 - x) ** 2 - 0.25)
        )

    return f


In [235]:
# zagięcie ma przechodzić przez y = 0.8 na okręgu o promieniu 0.5 wokół (0.5,0.5)
f = make_fold_function_through_y(s=0.5, side="right")
f_ = lambda x, y: 1*f(x, y) + f_1(x, y)
plot_f_3d_interactive(f, xlim=(0, 1), ylim=(0, 1), n=250, title="Fold through y=0.8")
